In [90]:
%matplotlib notebook
import warnings
warnings.filterwarnings("ignore")
import networkx as nx
import math
import random

def minCutKarger(G):
    # Create a local copy of the undirected graph
    G = G.copy()

    # Re-seed our random number generator, to ensure a different contraction.
    random.seed()
    while G.number_of_nodes() > 2:
        # Pick an edge at random
        idx = int((random.random() * (G.number_of_edges() - 1)))
        e =nx.edges(G)[idx]
        if G.number_of_nodes() == 5:
            print("Edge: ", e)
            print(G.edges(e[0]))
            print(G.edges(e[1]))
#         a, b = e[0], e[1]
        # Contract the two vertices
#         print("-----Contracting: ", a, b)
        G = nx.contracted_edge(G, e, self_loops=False)            
        if G.number_of_nodes() == 4:
            print("After contraction: ", G.edges(e[0]))
            print("---Edges")
            print(G.edges())
            print("---Nodes")
            print(G.nodes())
            print("----------------")
#         print("NEW SIZE: ", G.number_of_nodes())
        
    return G

G = nx.Graph()
for line in open("04_MinCut.txt"):
    row = line.strip().split("\t")
    sample = [int(x) for x in row]
    v = sample[0]
    for y in sample[1:]:
        G.add_edge(v, y)

n = nx.number_of_nodes(G)
iterations_needed = int((n**2)*math.log(n))
print("Iterations required to have a low chance of failure: ", iterations_needed)

# nx.draw(G)
cut = minCutKarger(G)
print(cut.nodes())
print(cut.edges())
print(cut.number_of_edges())


Iterations required to have a low chance of failure:  211932
Edge:  (79, 90)
[(79, 1), (79, 78), (79, 90), (79, 47)]
[(90, 79), (90, 78)]
After contraction:  [(79, 1), (79, 78), (79, 47)]
---Edges
[(1, 79), (1, 78), (79, 78), (79, 47), (78, 47)]
---Nodes
[1, 79, 78, 47]
----------------
[1, 47]
[(1, 47)]
1
